# Data-driven emission quantification of hot spots
This notebook demonstrates how to quantify CO$_2$ and NO$_x$ emissions from point sources using synthetic CO2M observations for a power plant and for a city.

In [ ]:
import os

import cartopy.crs as ccrs
import numpy as np
import pandas as pandas
import xarray

# import and setup matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

import ucat
import ddeq

Define coordinate reference systems used by the tool, the SMARTCARB model domain and the path to SMARTCARB data:

In [ ]:
# DHDN / Soldner Berlin
CRS = ccrs.epsg(3068)
WGS84 = ccrs.PlateCarree()

DOMAIN = ddeq.misc.Domain.from_nml(os.path.join(os.path.dirname(ddeq.__file__), 'data', 'INPUT_ORG'))

ROOT = '/project/coco2/fileshare/WP4/SMARTCARB/'

EMISSIONS_DATA_PATH = os.path.join(ROOT, 'timeseries')
SMARTCARB_DATA_PATH = os.path.join(ROOT, 'level2')
SMARTCARB_MODEL_PATH = os.path.join(ROOT, 'COSMO')

Read list of point sources in the SMARTCARB model domain from "sources.csv" file. The format of the `xarray.Dataset` is used by plume detection and emission quantification code internally to identify the point sources. 

In [ ]:
# list of point sources
sources = ddeq.misc.read_point_sources()
sources

## Synthetic satellite observations

Synthetic satellite observations are available from the SMARTCARB project (https://doi.org/10.5281/zenodo.4048227). The `ddeq` package can read the data files and automatically applies random noise and cloud filters to the observations. The code also fixes some issues with the dataset such as wrong emissions for industry in Berlin in January and July. It is also possible to scale the anthropogenic model tracers: 

In [ ]:
filename = os.path.join(SMARTCARB_DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')
data_level2 = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='medium', no2_noise_scenario='high')
data_level2

The data can easily be plotted using the `ddeq.vis.visualize` function, which requires the satellite data, the name of the trace gas, the SMARTCARB model domain, and the dataset of sources for labeling point sources. It is also possible to show the wind direction on the maker, which however requires ERA5 data.

In [ ]:
fig = ddeq.vis.visualize(data_level2, trace_gas='NO2', domain=DOMAIN, sources=sources, add_wind=False)

It is also possible to read and visualize the (vertically integrated) COSMO-GHG fields:

In [ ]:
time = pandas.Timestamp(data_level2.time.to_pandas())
filename = os.path.join(SMARTCARB_MODEL_PATH.format(start_year=2015, start_month=1 if time.month < 7 else 7),
                        time.strftime('cosmo_2d_%Y%m%d%H.nc'))

data_cosmo = ddeq.smartcarb.read_cosmo(filename, 'CO2')

fig = ddeq.vis.make_field_map(data_cosmo, trace_gas='CO2', domain=DOMAIN, vmin=404, vmax=408,
                              alpha=data_cosmo['CLCT'], border=50.0, label='XCO$_2$ [ppm]')

### Exercise 
* Read SMARTCARB Level-2 data for 23 April 2015, 11 UTC (orbit: 1670, lon_eq: 0483) using a low-noise CO2 and high-noise NO2 uncertainty scenario.
* Plot the XCO2 observations using `ddeq.vis.visualize`, mask cloud fractions larger than 1%, and add labels point sources (Berlin, Boxberg, Janschwalde, Lippendorf, Schwarze Pumpe and Turow).
* Read and add the XCO2 field from the COSMO-GHG model (`ddeq.smartcarb.read_trace_gas_field`) and additional fields (`ddeq.smartcarb.read_fields`) to the plot.
* Add a square showing the study area given by lower left and upper right points of 12.0°N, 50.7°E and 15.5°N, 52.7°N, respectively.

In [ ]:
# add your code here

## Plume detection algorithm
Plumes are regions how satellite pixels where CO2/NO2 values are significantly enhanced above the background
\begin{equation}
SNR = \frac{X - X_\mathrm{bg}}{\sqrt{\sigma_\mathrm{random}^2 + \sigma_\mathrm{sys}^2}} \geq z_\mathrm{thr}
\end{equation}
The value $X$ is computed by applying a Gaussian filter (other filters are possible) with size `filter_size` (default: 0.5 pixels). The background $X_\mathrm{bg}$ is computed using a median filter (size = 100 pixels). The threshold $z_\mathrm{thr}$ is computed for z-statistics using a probability $q$ (default: 0.99). Pixels for which above equation is true, are connected to regions using a labeling algorithm considering (horizontal, vertical and diagonal neighbors). Regions that overlap that are within the radius defined in `sources` of a point sources are assigned to the source. A region can be assigned to more than one source (overlapping plumes).

In [ ]:
filename = os.path.join(SMARTCARB_DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')
data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='low', no2_noise_scenario='high', co_noise_scenario='low', only_observations=False)

In [ ]:
data = ddeq.dplume.detect_plumes(data, sources, variable='NO2', variable_std='NO2_std',
                                 filter_type='gaussian', filter_size=0.5)

data.attrs.update(
    DESCRIPTION = ('Synthetic XCO2 and NO2 satellite image with ',
                   'auxiliary data for estimating CO2/NOx ',
                   'emissions of cities and point sources'),
    DATAORIGIN = 'SMARTCARB study',
    CREATOR = 'Gerrit Kuhlmann',
    EMAIL = 'gerrit.kuhlmann@empa.ch',
    AFFILIATION = 'Empa Duebendorf, Switzerland',
    DATE_CREATED = pandas.Timestamp.now().strftime('%Y-%m-%d %H:%M'),
)

The code computes several new fields that are added to the provided `data` dataset. The detected plumes are stored in the `detected_plume` data array (dims: nobs, nrows, source) where the length of source is equal to the number of detected plumes. 

In [ ]:
data

The plume detection can be visualized using the `ddeq.vis.visualize` function:

In [ ]:
ddeq.vis.visualize(data, trace_gas='NO2', domain=DOMAIN, add_wind=True, do_zoom=False, show_clouds=True);

### Exercise 
* Detect emission plumes Berlin and Jänschwalde using low-noise CO2 observations.
* Increase the size of the Gaussian filter to increase number of detected pixels.
* Visualize the results using `ddeq.vis.visualize`

In [ ]:
# add your code here

## Center lines and polygons

To estimate emissions for detected plumes, the following code fits a center curve for each detected plume. The code also adds across- and along-plume coordinates (`xp` and `yp`). The computation of plume coordinates can result in multiple solutions when the center line is strongly curved or the plume is small.

In [ ]:
filename = os.path.join(SMARTCARB_DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')
data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='low', no2_noise_scenario='high', 
                                  co_noise_scenario='low', only_observations=False)

data = ddeq.dplume.detect_plumes(data, sources.sel(source=['Berlin', 'Janschwalde']),
                                 variable='NO2', variable_std='NO2_std',
                                 filter_type='gaussian', filter_size=0.5)

ddeq.vis.visualize(data, trace_gas='NO2', domain=DOMAIN, add_wind=True, do_zoom=True, show_clouds=True);

In [ ]:
data, curves = ddeq.plume_coords.compute_plume_line_and_coords(data, crs=CRS, plume_area=25e3)

The following code shows the result:

In [ ]:
ddeq.vis.show_detected_plumes(data, curves, 'NO2', domain=DOMAIN, add_wind=True, do_zoom=True, crs=CRS);

## Prepare mass-balance approach

To prepare for the mass-balance approach the following code computes the CO2/NO2 background field, the plume signals and converts to mass columns in kg/m² using the `ucat` Python package. 

In [ ]:
data = ddeq.emissions.prepare_data(data, 'CO2')
data = ddeq.emissions.prepare_data(data, 'NO2')

It is possible to visualize different variables using the `variable` parameter:

In [ ]:
ddeq.vis.visualize(data, trace_gas='CO2', domain=DOMAIN, add_wind=False, do_zoom=True, show_clouds=True,
                   variable='CO2_minus_estimated_background_mass', vmin=-20e-3, vmax=60e-3,
                   label='CO$_2$ column density [kg m$^{-2}$]');

## Mass-balance approach
The following code estimated CO2 and NOx emissions for a point source (Jänschwalde) and a city (Berlin):

### Mass-balance for point source
First, we select subset of the dataset for specific source:

In [ ]:
source = 'Janschwalde'
this = ddeq.misc.select_source(data, source=source)

Wind speed and direction are taken from ERA5 data that are downloaded from the Copernicus Climate Data Store (CDS). This can be done automatically using `cdsapi` but requires a CDS account and it might be slow. At the moment, wind speed and direction are averaged between 800 and 1000 hPa.

In [ ]:
time = pandas.Timestamp(data.time.values)
wind_speed, wind_direction = ddeq.wind.get_wind(time, this['lon_o'], this['lat_o'], '.', components='ud')
wind_direction

The mass-balance approach is performed by the following function:

In [ ]:
results = ddeq.emissions.estimate(this, curves[source], source, trace_gases=['CO2', 'NO2'], 
                                  wind_speed=float(wind_speed), wind_direction=float(wind_direction),
                                  method='mass-balance', crs=CRS)

results

The results can be visualized with the following function:

In [ ]:
ddeq.vis.plot_mass_balance(data, curves, results, source=source, domain=DOMAIN, crs=CRS,
                           data_path=EMISSIONS_DATA_PATH);

### Mass-balance for city
The mass-balance approach over a city is slightly different, because the flux slowly builds up over the city area. For NO$_x$, fluxes over the city are therefore modeled by a Gaussian curve.

In [ ]:
source = 'Berlin'
this = ddeq.misc.select_source(data, source=source)

time = pandas.Timestamp(data.time.values)
wind_speed, wind_direction = ddeq.wind.get_wind(time, this.lon_o, this.lat_o, data_path='.', components='vd')

results_berlin = ddeq.emissions.estimate(this, curves[source], source, trace_gases=['CO2', 'NO2'], 
                                  wind_speed=float(wind_speed), wind_direction=float(wind_direction),
                                  method='mass-balance', crs=CRS)

In [ ]:
ddeq.vis.plot_mass_balance(data, curves, results_berlin, source='Berlin', domain=DOMAIN, crs=CRS, 
                           data_path=EMISSIONS_DATA_PATH);

### Exercise
Write code to quantify the CO2 and NOx emission another point source.

In [ ]:
# add your code here